In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/ && pip3 install -r requirements.txt

%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages

!mkdir /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/data
!mv /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data/data.tfrecord /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/data/data.tfrecord

In [6]:

# install waymo dataset utils in utils
# https://github.com/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb
!cd utils && rm -rf waymo-od > /dev/null
!cd utils && git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd utils/waymo-od && git branch -a
!cd utils/waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

# directly retrieve data from google
# https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041
'''
from google.colab import auth
auth.authenticate_user()
project_id = 'nifty-depth-246308'
!gcloud config set project {project_id}
!gsutil ls
bucket_name = 'medium_demo_bucket_190710'
!gsutil -m cp -r /content/mnt/My\ Drive/Colab\ Notebooks/DeepCV_Packages/* gs://{bucket_name}/
'''

/bin/bash: line 0: cd: too many arguments
[Errno 2] No such file or directory: 'content/mnt/My Drive/Colab Notebooks/DeepCV_Packages'
/content
mkdir: cannot create directory ‘content/mnt/My Drive/Colab Notebooks/DeepCV_Packages/DeepCV_Lab/data’: No such file or directory
mv: cannot stat 'content/mnt/My Drive/Colab Notebooks/DeepCV_Packages/data/data.tfrecord': No such file or directory
/bin/bash: line 0: cd: utils: No such file or directory
/bin/bash: line 0: cd: utils: No such file or directory
/bin/bash: line 0: cd: utils/waymo-od: No such file or directory
/bin/bash: line 0: cd: utils/waymo-od: No such file or directory
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)


"\nfrom google.colab import auth\nauth.authenticate_user()\nproject_id = 'nifty-depth-246308'\n!gcloud config set project {project_id}\n!gsutil ls\nbucket_name = 'medium_demo_bucket_190710'\n!gsutil -m cp -r /content/mnt/My\\ Drive/Colab\\ Notebooks/DeepCV_Packages/* gs://{bucket_name}/\n"